In [5]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os

num_epochs = 1000
batch_size = 32
learning_rate = 0.01

In [6]:
def _decode_and_resize(filename, label):
    image_string = tf.io.read_file(filename)            # 读取原始文件
    image_decoded = tf.image.decode_jpeg(image_string)  # 解码JPEG图片
    image_resized = tf.image.resize(image_decoded, [224, 224]) / 255.0
    return image_resized, label

data_text = pd.read_csv('train_data.csv')
# 构建训练数据集
train_filenames = tf.constant([filename for filename in list(data_text.path.values)])
train_labels = list(data_text.page_num.values)

train_dataset = tf.data.Dataset.from_tensor_slices((train_filenames, train_labels))
train_dataset = train_dataset.map(
    map_func=_decode_and_resize, 
    num_parallel_calls=tf.data.experimental.AUTOTUNE)
# 取出前buffer_size个数据放入buffer，并从其中随机采样，采样后的数据用后续数据替换
train_dataset = train_dataset.shuffle(buffer_size=500)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [7]:
resnet50_fine_tune = tf.keras.models.Sequential()

resnet50_fine_tune.add(tf.keras.applications.ResNet50(include_top = False, pooling = 'avg', weights = 'imagenet'))
resnet50_fine_tune.add(tf.keras.layers.Dense(48, activation = 'softmax'))
resnet50_fine_tune.layers[0].trainable = False
# resnet50_fine_tune.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=['accuracy'])
resnet50_fine_tune.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=[tf.keras.metrics.sparse_categorical_accuracy]
)
resnet50_fine_tune.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 48)                98352     
Total params: 23,686,064
Trainable params: 98,352
Non-trainable params: 23,587,712
_________________________________________________________________


In [8]:
resnet50_fine_tune.fit(train_dataset, epochs=num_epochs)

Epoch 1/1000
61/61 [==============================] - 3s 48ms/step - loss: 4.6815 - sparse_categorical_accuracy: 0.0196
Epoch 2/1000
61/61 [==============================] - 3s 45ms/step - loss: 4.4460 - sparse_categorical_accuracy: 0.0279
Epoch 3/1000
61/61 [==============================] - 3s 43ms/step - loss: 4.3865 - sparse_categorical_accuracy: 0.0268
Epoch 4/1000
61/61 [==============================] - 3s 43ms/step - loss: 4.2949 - sparse_categorical_accuracy: 0.0377
Epoch 5/1000
61/61 [==============================] - 3s 45ms/step - loss: 4.2633 - sparse_categorical_accuracy: 0.0418
Epoch 6/1000
61/61 [==============================] - 3s 46ms/step - loss: 4.2234 - sparse_categorical_accuracy: 0.0542
Epoch 7/1000
61/61 [==============================] - 3s 44ms/step - loss: 4.1133 - sparse_categorical_accuracy: 0.0547
Epoch 8/1000
61/61 [==============================] - 3s 46ms/step - loss: 4.1734 - sparse_categorical_accuracy: 0.0624
Epoch 9/1000
61/61 [====================

61/61 [==============================] - 3s 46ms/step - loss: 3.0630 - sparse_categorical_accuracy: 0.2456
Epoch 69/1000
61/61 [==============================] - 3s 48ms/step - loss: 3.0340 - sparse_categorical_accuracy: 0.2621
Epoch 70/1000
61/61 [==============================] - 3s 47ms/step - loss: 2.9923 - sparse_categorical_accuracy: 0.2611
Epoch 71/1000
61/61 [==============================] - 3s 48ms/step - loss: 2.9521 - sparse_categorical_accuracy: 0.2781
Epoch 72/1000
61/61 [==============================] - 3s 48ms/step - loss: 2.8988 - sparse_categorical_accuracy: 0.2797
Epoch 73/1000
61/61 [==============================] - 3s 47ms/step - loss: 3.0184 - sparse_categorical_accuracy: 0.2652
Epoch 74/1000
61/61 [==============================] - 3s 48ms/step - loss: 2.9865 - sparse_categorical_accuracy: 0.2755
Epoch 75/1000
61/61 [==============================] - 3s 46ms/step - loss: 2.9581 - sparse_categorical_accuracy: 0.2833
Epoch 76/1000
61/61 [=========================

61/61 [==============================] - 3s 48ms/step - loss: 2.4543 - sparse_categorical_accuracy: 0.3901
Epoch 136/1000
61/61 [==============================] - 3s 48ms/step - loss: 2.5937 - sparse_categorical_accuracy: 0.3602
Epoch 137/1000
61/61 [==============================] - 3s 47ms/step - loss: 2.5563 - sparse_categorical_accuracy: 0.3638
Epoch 138/1000
61/61 [==============================] - 3s 47ms/step - loss: 2.5316 - sparse_categorical_accuracy: 0.3669
Epoch 139/1000
61/61 [==============================] - 3s 48ms/step - loss: 2.4871 - sparse_categorical_accuracy: 0.3736
Epoch 140/1000
61/61 [==============================] - 3s 48ms/step - loss: 2.4500 - sparse_categorical_accuracy: 0.3937
Epoch 141/1000
61/61 [==============================] - 3s 47ms/step - loss: 2.4585 - sparse_categorical_accuracy: 0.3824
Epoch 142/1000
61/61 [==============================] - 3s 48ms/step - loss: 2.3905 - sparse_categorical_accuracy: 0.4082
Epoch 143/1000
61/61 [=================

61/61 [==============================] - 3s 47ms/step - loss: 2.1147 - sparse_categorical_accuracy: 0.4763
Epoch 203/1000
61/61 [==============================] - 3s 48ms/step - loss: 2.0871 - sparse_categorical_accuracy: 0.4778
Epoch 204/1000
61/61 [==============================] - 3s 48ms/step - loss: 2.1215 - sparse_categorical_accuracy: 0.4685
Epoch 205/1000
61/61 [==============================] - 3s 49ms/step - loss: 2.0567 - sparse_categorical_accuracy: 0.4768
Epoch 206/1000
61/61 [==============================] - 3s 48ms/step - loss: 2.0376 - sparse_categorical_accuracy: 0.4809
Epoch 207/1000
61/61 [==============================] - 3s 49ms/step - loss: 2.0805 - sparse_categorical_accuracy: 0.4727
Epoch 208/1000
61/61 [==============================] - 3s 48ms/step - loss: 2.0929 - sparse_categorical_accuracy: 0.4675
Epoch 209/1000
61/61 [==============================] - 3s 49ms/step - loss: 2.1543 - sparse_categorical_accuracy: 0.4665
Epoch 210/1000
61/61 [=================

61/61 [==============================] - 3s 49ms/step - loss: 1.8379 - sparse_categorical_accuracy: 0.5402
Epoch 270/1000
61/61 [==============================] - 3s 49ms/step - loss: 1.8255 - sparse_categorical_accuracy: 0.5279
Epoch 271/1000
61/61 [==============================] - 3s 49ms/step - loss: 1.8842 - sparse_categorical_accuracy: 0.5088
Epoch 272/1000
61/61 [==============================] - 3s 49ms/step - loss: 1.8205 - sparse_categorical_accuracy: 0.5253
Epoch 273/1000
61/61 [==============================] - 3s 49ms/step - loss: 1.7884 - sparse_categorical_accuracy: 0.5506
Epoch 274/1000
61/61 [==============================] - 3s 49ms/step - loss: 1.7617 - sparse_categorical_accuracy: 0.5402
Epoch 275/1000
61/61 [==============================] - 3s 49ms/step - loss: 1.7698 - sparse_categorical_accuracy: 0.5480
Epoch 276/1000
61/61 [==============================] - 3s 49ms/step - loss: 1.7598 - sparse_categorical_accuracy: 0.5526
Epoch 277/1000
61/61 [=================

...

61/61 [==============================] - 3s 49ms/step - loss: 0.9683 - sparse_categorical_accuracy: 0.7456
Epoch 670/1000
61/61 [==============================] - 3s 48ms/step - loss: 0.8951 - sparse_categorical_accuracy: 0.7652
Epoch 671/1000
61/61 [==============================] - 3s 48ms/step - loss: 0.9271 - sparse_categorical_accuracy: 0.7647
Epoch 672/1000
61/61 [==============================] - 3s 49ms/step - loss: 0.9106 - sparse_categorical_accuracy: 0.7632
Epoch 673/1000
61/61 [==============================] - 3s 49ms/step - loss: 0.9752 - sparse_categorical_accuracy: 0.7461
Epoch 674/1000
61/61 [==============================] - 3s 49ms/step - loss: 0.9741 - sparse_categorical_accuracy: 0.7379
Epoch 675/1000
61/61 [==============================] - 3s 48ms/step - loss: 0.8933 - sparse_categorical_accuracy: 0.7735
Epoch 676/1000
61/61 [==============================] - 3s 48ms/step - loss: 0.9202 - sparse_categorical_accuracy: 0.7632
Epoch 677/1000
61/61 [=================

61/61 [==============================] - 3s 49ms/step - loss: 0.8737 - sparse_categorical_accuracy: 0.7755
Epoch 737/1000
61/61 [==============================] - 3s 49ms/step - loss: 0.9206 - sparse_categorical_accuracy: 0.7575
Epoch 738/1000
61/61 [==============================] - 3s 49ms/step - loss: 0.8582 - sparse_categorical_accuracy: 0.7776
Epoch 739/1000
61/61 [==============================] - 3s 48ms/step - loss: 0.9791 - sparse_categorical_accuracy: 0.7528
Epoch 740/1000
61/61 [==============================] - 3s 48ms/step - loss: 0.9543 - sparse_categorical_accuracy: 0.7456
Epoch 741/1000
61/61 [==============================] - 3s 49ms/step - loss: 0.8638 - sparse_categorical_accuracy: 0.7632
Epoch 742/1000
61/61 [==============================] - 3s 48ms/step - loss: 0.8816 - sparse_categorical_accuracy: 0.7673
Epoch 743/1000
61/61 [==============================] - 3s 48ms/step - loss: 0.8288 - sparse_categorical_accuracy: 0.7828
Epoch 744/1000
61/61 [=================

61/61 [==============================] - 3s 48ms/step - loss: 0.7954 - sparse_categorical_accuracy: 0.7967
Epoch 804/1000
61/61 [==============================] - 3s 48ms/step - loss: 0.7888 - sparse_categorical_accuracy: 0.7884
Epoch 805/1000
61/61 [==============================] - 3s 48ms/step - loss: 0.8372 - sparse_categorical_accuracy: 0.7812
Epoch 806/1000
61/61 [==============================] - 3s 48ms/step - loss: 0.8800 - sparse_categorical_accuracy: 0.7812
Epoch 807/1000
61/61 [==============================] - 3s 48ms/step - loss: 0.8396 - sparse_categorical_accuracy: 0.7879
Epoch 808/1000
61/61 [==============================] - 3s 48ms/step - loss: 0.8119 - sparse_categorical_accuracy: 0.7833
Epoch 809/1000
61/61 [==============================] - 3s 48ms/step - loss: 0.7862 - sparse_categorical_accuracy: 0.8003
Epoch 810/1000
61/61 [==============================] - 3s 49ms/step - loss: 0.7446 - sparse_categorical_accuracy: 0.8168
Epoch 811/1000
61/61 [=================

61/61 [==============================] - 3s 49ms/step - loss: 0.7389 - sparse_categorical_accuracy: 0.8039
Epoch 871/1000
61/61 [==============================] - 3s 49ms/step - loss: 0.8128 - sparse_categorical_accuracy: 0.7946
Epoch 872/1000
61/61 [==============================] - 3s 48ms/step - loss: 0.6523 - sparse_categorical_accuracy: 0.8447
Epoch 873/1000
61/61 [==============================] - 3s 48ms/step - loss: 0.7210 - sparse_categorical_accuracy: 0.8050
Epoch 874/1000
61/61 [==============================] - 3s 49ms/step - loss: 0.6980 - sparse_categorical_accuracy: 0.8240
Epoch 875/1000
61/61 [==============================] - 3s 49ms/step - loss: 0.7382 - sparse_categorical_accuracy: 0.8044
Epoch 876/1000
61/61 [==============================] - 3s 48ms/step - loss: 0.7029 - sparse_categorical_accuracy: 0.8199
Epoch 877/1000
61/61 [==============================] - 3s 48ms/step - loss: 0.6906 - sparse_categorical_accuracy: 0.8173
Epoch 878/1000
61/61 [=================

61/61 [==============================] - 3s 47ms/step - loss: 0.7438 - sparse_categorical_accuracy: 0.8029
Epoch 938/1000
61/61 [==============================] - 3s 48ms/step - loss: 0.6376 - sparse_categorical_accuracy: 0.8375
Epoch 939/1000
61/61 [==============================] - 3s 49ms/step - loss: 0.6625 - sparse_categorical_accuracy: 0.8297
Epoch 940/1000
61/61 [==============================] - 3s 49ms/step - loss: 0.6752 - sparse_categorical_accuracy: 0.8153
Epoch 941/1000
61/61 [==============================] - 3s 49ms/step - loss: 0.6783 - sparse_categorical_accuracy: 0.8163
Epoch 942/1000
61/61 [==============================] - 3s 49ms/step - loss: 0.8144 - sparse_categorical_accuracy: 0.7781
Epoch 943/1000
61/61 [==============================] - 3s 49ms/step - loss: 0.7418 - sparse_categorical_accuracy: 0.8008
Epoch 944/1000
61/61 [==============================] - 3s 49ms/step - loss: 0.7254 - sparse_categorical_accuracy: 0.8034
Epoch 945/1000
61/61 [=================